In [1]:
%run ./db_variables.ipynb

In [2]:
import psycopg2
import os
import datetime

In [4]:
# Add receipt_pred table

# Database connection details

try:
    # Establish a connection to the database
    connection = psycopg2.connect(
        host=db_host,
        port=db_port,
        user=db_user,
        password=db_password,
        database = db_name
    )

    with connection.cursor() as cursor:
        # SQL query to create a table
        create_table_query = """
        CREATE TABLE receipt_pred (
            receipt_id VARCHAR(255) PRIMARY KEY NOT NULL UNIQUE,
            user_id VARCHAR(255) NOT NULL,
            pred_amount DECIMAL(10,2) NOT NULL,
            pred_date DATE NOT NULL,
            pred_category VARCHAR(255) NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        
        # Execute the query
        cursor.execute(create_table_query)
        print("Table 'receipt_pred' created successfully!")

    # Commit the changes
    connection.commit()

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the connection
    if connection:
        connection.close()

Table 'receipts' created successfully!


In [3]:
# Add receipt_pred table

# Database connection details

try:
    # Establish a connection to the database
    connection = psycopg2.connect(
        host=db_host,
        port=db_port,
        user=db_user,
        password=db_password,
        database = db_name
    )

    with connection.cursor() as cursor:
        # SQL query to create a table
        create_table_query = """
        CREATE TABLE receipt_data (
            receipt_id VARCHAR(255) PRIMARY KEY NOT NULL UNIQUE,
            user_id VARCHAR(255) NOT NULL,
            act_amount DECIMAL(10,2) NOT NULL,
            act_date DATE NOT NULL,
            act_category VARCHAR(255) NOT NULL,
            comment VARCHAR(255) DEFAULT NULL,
            report_number VARCHAR(45) DEFAULT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        
        # Execute the query
        cursor.execute(create_table_query)
        print("Table 'receipt_data' created successfully!")

    # Commit the changes
    connection.commit()

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the connection
    if connection:
        connection.close()

Error: relation "receipt_data" already exists



In [2]:
# Add accounts table


try:

    connection = psycopg2.connect(
        host=db_host,
        port=db_port,
        user=db_user,
        password=db_password,
        database = db_name
    )

    with connection.cursor() as cursor:
 
        create_table_query = """
        CREATE TABLE accounts (
            category VARCHAR(255) PRIMARY KEY NOT NULL UNIQUE,
            account VARCHAR(255) DEFAULT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        
 
        cursor.execute(create_table_query)
        print("Table 'accounts' created successfully!")


    connection.commit()

except Exception as e:
    print(f"Error: {e}")

finally:
 
    if connection:
        connection.close()

Table 'accounts' created successfully!


In [5]:
# Add report_data table

try:

    connection = psycopg2.connect(
        host=db_host,
        port=db_port,
        user=db_user,
        password=db_password,
        database = db_name
    )

    with connection.cursor() as cursor:
    
        create_table_query = """
        CREATE TABLE report_data (
            user_id VARCHAR(255) NOT NULL UNIQUE,
            report_number VARCHAR(45) NOT NULL,
            submitted BOOLEAN NOT NULL DEFAULT FALSE,
            approved BOOLEAN NOT NULL DEFAULT FALSE,
            returned BOOLEAN NOT NULL DEFAULT FALSE,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            PRIMARY KEY (report_number, user_id)
        );
        """
        

        cursor.execute(create_table_query)
        print("Table 'report_data' created successfully!")


    connection.commit()

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the connection
    if connection:
        connection.close()

Table 'report_data' created successfully!


In [16]:
def lambda_handler(event, db_host, db_port, db_user, db_password, db_name ):


    # Table and data
    table_name = "receipt_pred"  
    receipt_id = event.get("receipt_id")
    user_id = event.get("user_id")
    pred_amount = event.get("pred_amt")
    pred_date_str = event.get("pred_date")  
    pred_category = event.get("pred_category")

    pred_date = datetime.datetime.strptime(pred_date_str, "%Y-%m-%d").date() if pred_date_str else None

    insert_query = """
    INSERT INTO receipt_pred (receipt_id, user_id, pred_amount, pred_date, pred_category)
    VALUES (%s, %s, %s, %s, %s)
    RETURNING user_id;
    """
    print("here")
    connection = None
    try:

        connection = psycopg2.connect(
            host=db_host,
            port=db_port,
            user=db_user,
            password=db_password,
            database = db_name
        )
        
        cursor = connection.cursor()

  
        cursor.execute(insert_query, (receipt_id, user_id, pred_amount, pred_date, pred_category))
        
     
        new_user_id = cursor.fetchone()[0]


        connection.commit()

        return {"statusCode": 200, "message": "Row inserted successfully!", "user_id": new_user_id}

    except Exception as e:
        return {"statusCode": 500, "error": str(e)}

    finally:
        if connection is not None:
            cursor.close()
            connection.close()
    print('END')

In [11]:
def lambda_handler2(event, db_host, db_port, db_user, db_password, db_name):

    # Table and data
    table_name = "receipt_data"  # Change to your table name
    receipt_id = event.get("receipt_id")
    user_id = event.get("user_id")
    act_amount = event.get("pred_amt")
    act_date_str = event.get("pred_date")  # Example format: "2025-01-24"
    act_category = event.get("pred_category")

    act_date = datetime.datetime.strptime(act_date_str, "%Y-%m-%d").date() if act_date_str else None

    insert_query = """
    INSERT INTO receipt_data (receipt_id, user_id, act_amount, act_date, act_category)
    VALUES (%s, %s, %s, %s, %s)
    RETURNING user_id;
    """
    print("here")
    connection = None
    try:
        # Connect to PostgreSQL database
        connection = psycopg2.connect(
            host=db_host,
            port=db_port,
            user=db_user,
            password=db_password,
            database = db_name
        )
        
        cursor = connection.cursor()

        # Execute the INSERT query
        cursor.execute(insert_query, (receipt_id, user_id, act_amount, act_date, act_category))
        
        # Get the auto-generated product ID
        new_user_id = cursor.fetchone()[0]

        # Commit the transaction
        connection.commit()

        return {"statusCode": 200, "message": "Row inserted successfully!", "user_id": new_user_id}

    except Exception as e:
        return {"statusCode": 500, "error": str(e)}

    finally:
        if connection is not None:
            cursor.close()
            connection.close()
    print('END')

In [12]:
event = {
    'receipt_id':'test2',
    'user_id':'id_test',
    'pred_amt': 123.00,
    'pred_date': '2025-02-06',
    'pred_category':'Meals'
}

In [13]:
lambda_handler2(event = event, db_host, db_port, db_user, db_password, db_name)

here


{'statusCode': 200,
 'message': 'Row inserted successfully!',
 'user_id': 'id_test'}

In [19]:
import psycopg2
import os

def lambda_handler2():


    query = "SELECT * FROM receipts;"
    connection = None
    try:
        connection = psycopg2.connect(
            host=db_host,
            port=db_port,
            user=db_user,
            password=db_password,
            database=db_name
        )
        cursor = connection.cursor()

        cursor.execute(query)
        results = cursor.fetchall()

        return {"statusCode": 200, "data": results}

    except Exception as e:
        return {"statusCode": 500, "error": str(e)}

    finally:
        if connection is not None:
            cursor.close()
            connection.close()


In [3]:
# Quickly drop tables

try:

    connection = psycopg2.connect(
        host=db_host,
        port=db_port,
        user=db_user,
        password=db_password,
        database = db_name
    )

    with connection.cursor() as cursor:

        create_table_query = """
        DROP TABLE IF EXISTS receipts;
        """
        

        cursor.execute(create_table_query)
        print("Table 'receipts' dropped successfully!")

    connection.commit()

except Exception as e:
    print(f"Error: {e}")

finally:
 
    if connection:
        connection.close()

Table 'receipts' dropped successfully!
